In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [8]:
bag_of_words_top_1000 = pd.read_feather('../../data/transform/bag_of_words_top_1000.feather')

In [5]:
def corpus_topics_top_words(model, features, no_top_words):
    topic_dict = {}
    for topic_idx, topic in enumerate(model.components_):
        topic_dict[topic_idx] = [features[i] for i in topic.argsort()[:-no_top_words - 1:-1]]
    return topic_dict

def song_topics(model, song):
    topic_dict = []
    for topic_idx, topic in enumerate(model.components_):
        topic_dict.append(sum(topic*song))
    return topic_dict

In [9]:
from sklearn.decomposition import LatentDirichletAllocation

mxm_25_top1000 = LatentDirichletAllocation(n_topics=25, random_state=0)
mxm_25_top1000.fit(bag_of_words_top_1000.iloc[:,12:])

/Users/spare/anaconda3/envs/py3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
/Users/spare/anaconda3/envs/py3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
             mean_change_tol=0.001, n_components=10, n_jobs=1, n_topics=25,
             perp_tol=0.1, random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [12]:
top_per_topic_words = corpus_topics_top_words(mxm_25_top1000,bag_of_words_top_1000.iloc[:,12:].columns.values, 10)

#save per/song topic results to df
song_topic_weights = np.zeros([len(bag_of_words_top_1000.iloc[:,12:]),25])
temp = bag_of_words_top_1000.iloc[:,12:]
for i in tqdm(range(len(mxm_dataset_1000.drop('track_id', axis=1)))):
    song_weights = pd.Series(song_topics(mxm_25_top1000, temp.iloc[i]))
    song_topic_weights[i] = song_weights


  0%|          | 311/237662 [00:01<21:00, 188.32it/s]
Exception in thread Thread-4:
Traceback (most recent call last):
  File "/Users/spare/anaconda3/envs/py3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/spare/anaconda3/envs/py3/lib/python3.6/site-packages/tqdm/_monitor.py", line 63, in run
    for instance in self.tqdm_cls._instances:
  File "/Users/spare/anaconda3/envs/py3/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

100%|██████████| 237662/237662 [17:52<00:00, 221.60it/s]


In [15]:
initial_topic_names = list(range(25))
song_topic_weights_df = pd.DataFrame(data =song_topic_weights, columns=initial_topic_names)
song_topic_weights_df['track_id'] = mxm_dataset_1000['track_id']
df_topic_weights_reduced_df = song_topic_weights_df[['track_id',11,15,20]]
df_topic_weights_reduced_df.columns = 'track_id', 'love','death','religion'

In [17]:
topic_words = {}
topic_words['love'] = top_per_topic_words[11]
topic_words['death'] = top_per_topic_words[15]
topic_words['religion'] = top_per_topic_words[20]
print(topic_words)

{'love': ['love', 'heart', 'sweet', 'true', 'give', 'enough', 'darling', 'touch', 'vision', 'found'], 'death': ['life', 'die', 'run', 'dead', 'kill', 'dream', 'blood', 'death', 'scream', 'clear'], 'religion': ['us', 'god', 'live', 'dance', 'people', 'heaven', 'hand', 'stand', 'angel', 'beautiful']}


In [18]:
bag_of_words_top_1000['love'] = df_topic_weights_reduced_df['love']
bag_of_words_top_1000['religion'] = df_topic_weights_reduced_df['religion']
bag_of_words_top_1000['death'] = df_topic_weights_reduced_df['death']

In [23]:
pd.DataFrame(topic_words).to_feather('../../data/transform/topic_words.feather')
bag_of_words_top_1000.to_feather('../../data/transform/bag_of_words_top_1000_topics.feather')